In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import List, Annotated
from io import BytesIO
from PIL import Image
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import fitz
import base64

In [ ]:
class PDF_Load:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path

    def extract_page(self):
        return extract_page(self.pdf_path)



In [ ]:
import fitz  # PyMuPDF
import base64
import openai
from io import BytesIO
from typing import List, Dict
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

class PDFProcessor:
    def __init__(self, pdf_path: str):
        """
        PDF를 처리하는 클래스
        :param pdf_path: PDF 파일 경로
        """
        self.pdf_path = pdf_path
        self.pdf_data = []  # 추출된 PDF 데이터 저장

    def convert_image_to_base64(self, image_bytes):
        """
        이미지 바이트를 base64 문자열로 변환
        """
        return base64.b64encode(image_bytes).decode('utf-8')

    def classify_image_type(self, image_bytes):
        """
        GPT-4o를 사용하여 이미지가 그래프/도표인지 분류
        Returns:
            bool: True if image is graph/chart, False otherwise
        """
        llm = ChatOpenAI(model="gpt-4o", max_tokens=1000)
        base64_image = self.convert_image_to_base64(image_bytes)

        prompt = (
            "다음 이미지를 아래 두 가지 중 하나로 분류하세요:\n"
            "1. 그래프/도표\n"
            "2. 그외\n"
            "반드시 위 중 하나의 항목만 정확히 출력하세요."
        )

        messages = [
            SystemMessage(content="당신은 이미지 분류 전문 AI 에이전트입니다."),
            HumanMessage(content=[
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ])
        ]

        response = llm.invoke(messages)
        label = response.content.strip()

        # 그래프/도표인 경우에만 True 반환
        return label == "그래프/도표"

    def get_total_image_area_ratio(self, page, images):
        """
        페이지에서 감지된 모든 이미지의 총 면적 비율을 계산.
        """
        total_ratio = 0
        for img in images:
            xref = img[0]
            img_rects = page.get_image_rects(xref)
            if img_rects:
                x0, y0, x1, y1 = img_rects[0]
                img_width = x1 - x0
                img_height = y1 - y0
                img_area = img_width * img_height

                page_width, page_height = page.rect.width, page.rect.height
                page_area = page_width * page_height

                total_ratio += img_area / page_area

        return total_ratio

    def should_interpret_image(self, image_bytes, total_area_ratio):
        """
        이미지 해석 여부를 결정하는 함수.
        Args:
            image_bytes: 이미지 바이트 데이터
            total_area_ratio (float): 페이지 내 모든 이미지의 총 면적 비율
        Returns:
            bool: 이미지를 해석해야 하면 True, 아니면 False
        """
        # 먼저 면적 비율 확인
        if total_area_ratio >= 0.5:
            return True

        # 면적 비율이 0.5 미만인 경우에만 LLM으로 그래프/도표 여부 확인
        is_graph_or_chart = self.classify_image_type(image_bytes)
        return is_graph_or_chart

    def extract_page(self):
        """
        PDF에서 텍스트 및 이미지 정보를 추출.
        Returns:
            list: 각 페이지별 정보를 담은 딕셔너리 리스트
                - page: 페이지 번호 (1부터 시작)
                - text: 페이지에서 추출된 전체 텍스트
                - images: 이미지 정보 리스트
                    - image: base64로 인코딩된 이미지
                    - image_type: 이미지 해석 필요 여부 (bool)
        """
        doc = fitz.open(self.pdf_path)
        total_pages = len(doc)
        pdf_data = []

        for page_num in range(total_pages):
            page = doc[page_num]
            text = page.get_text("text")
            images = page.get_images(full=True)
            image_data = []

            # 페이지의 총 이미지 면적 비율 계산
            total_area_ratio = self.get_total_image_area_ratio(page, images)

            for img in images:
                xref = img[0]
                img_rects = page.get_image_rects(xref)

                if img_rects:
                    # 이미지 추출
                    pix = fitz.Pixmap(doc, xref)
                    if pix.n > 4:  # CMYK를 RGB로 변환
                        pix = fitz.Pixmap(fitz.csRGB, pix)

                    # 이미지를 바이트로 변환
                    img_bytes = pix.tobytes()
                    img_base64 = self.convert_image_to_base64(img_bytes)

                    # 이미지 해석 여부 결정 (면적 비율 먼저 확인 후 필요시 LLM 사용)
                    should_interpret = self.should_interpret_image(img_bytes, total_area_ratio)

                    image_data.append({
                        "image": img_base64,
                        "image_type": should_interpret
                    })

                    pix = None  # 메모리 해제

            pdf_data.append({
                "page": page_num + 1,
                "text": text,
                "images": image_data
            })

        self.pdf_data = pdf_data  # 클래스 인스턴스 변수에 저장
        return pdf_data

    def filter_important_images(self):
        """
        PDF 데이터에서 해석이 필요한 이미지(image_type이 True)만 필터링하여 반환
        페이지는 모두 유지하고 이미지만 필터링합니다.

        Returns:
            list: 필터링된 PDF 데이터 (모든 페이지 포함, 해석이 필요한 이미지만 포함)
        """
        filtered_data = []

        for page_info in self.pdf_data:
            # image_type이 True인 이미지만 필터링
            important_images = [img for img in page_info['images'] if img['image_type']]

            # 모든 페이지를 포함하되, 필터링된 이미지만 포함
            filtered_data.append({
                "page": page_info['page'],
                "text": page_info['text'],
                "images": important_images  # 빈 리스트가 될 수도 있음
            })

        return filtered_data

In [ ]:
class Generate_Script:
    def __init__(self, pdf_data, full_document):
        self.pdf_data = pdf_data
        self.full_document = full_document
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
        self.vision_llm = ChatOpenAI(model="gpt-4v-turbo", temperature=0.0)
        self.previous_script: List[str] = []  # 이전 대본을 저장하는 리스트

    def image_analysis(self, page_idx: int, full_document: str) -> str:
        """
        이미지 분석 메서드
        :param page_idx: 현재 페이지 인덱스
        :param full_document: 전체 문서 문자열
        :return: 이미지 분석 결과 문자열
        """
        page_data = self.pdf_data[page_idx]
        images = page_data["images"]
        image_descriptions = []

        for image in images:
            if image["image_type"]: 
                template = """
                다음은 발표 자료에 대한 정보 입니다:
                - 발표 자료에 대한 전체 문서: {full_document}
                - 현재 페이지 텍스트: {page_data["text"]}
                - 현재 페이지 이미지: {image["image"]}

                이를 바탕으로 이미지에 대한 설명을 작성해주세요.
                """ 
                prompt = PromptTemplate.from_template(template)
                image_desc = self.vision_llm.invoke(prompt.format(full_document=full_document, page_data=page_data, image=image))
                image_descriptions.append(image_desc)

        return '\n'.join(image_descriptions) if image_descriptions else ""

    def generate_script(self, page_idx: int, full_document: str) -> str:
        """
        발표 대본 생성 메서드
        :param page_idx: 현재 페이지 인덱스
        :param full_document: 전체 문서 문자열
        :return: 생성된 발표 대본 문자열
        """ 
        page_data = self.pdf_data[page_idx]
        text = page_data["text"]

        image_descriptions = self.image_analysis(page_idx, full_document)

        # 최근 3개의 이전 대본을 참고하도록 설정
        previous_script_text = "\n".join(self.previous_script[-3:]) if self.previous_script else "없음"

        template = """
        다음은 발표 자료에 대한 정보 입니다:
        - 발표 자료에 대한 전체 문서: {full_document}
        - 현재 페이지 텍스트: {text}
        - 현재 페이지 이미지: {image_descriptions}
        - 이전 페이지 발표 대본: {previous_script}

        위 정보를 바탕으로 발표자가 자연스럽게 발표할 수 있도록 대본을 작성해주세요.
        발표 대본은 논리적인 흐름을 유지하고, 강조해야 할 핵심 내용을 포함해야 합니다.
        """     
        prompt = PromptTemplate.from_template(template)
        script = self.llm.invoke(prompt.format(full_document=full_document, text=text, image_descriptions=image_descriptions, previous_script=previous_script_text))

        # 생성된 발표 대본을 이전 대본 리스트에 저장
        self.previous_script.append(script)

        return script

In [ ]:
import fitz  # PyMuPDF
import base64
import openai
from io import BytesIO
from typing import List, Dict
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import PromptTemplate


# 부모 클래스: PDF에서 텍스트 & 이미지 추출
class PDFProcessor:
    def __init__(self, pdf_path: str):
        """
        PDF에서 텍스트 및 이미지 정보를 추출하는 기본 클래스
        :param pdf_path: PDF 파일 경로
        """
        self.pdf_path = pdf_path
        self.pdf_data = []  # 추출된 PDF 데이터 저장

    def extract_page(self):
        """
        PDF에서 텍스트 및 이미지 정보를 추출.
        """
        doc = fitz.open(self.pdf_path)
        total_pages = len(doc)
        pdf_data = []

        for page_num in range(total_pages):
            page = doc[page_num]
            text = page.get_text("text")
            images = page.get_images(full=True)
            image_data = []

            for img in images:
                xref = img[0]
                img_rects = page.get_image_rects(xref)

                if img_rects:
                    img_bytes = self.extract_image_bytes(doc, xref)
                    img_base64 = self.convert_image_to_base64(img_bytes)

                    image_data.append({
                        "image": img_base64,
                        "image_bytes": img_bytes,
                        "bbox": img_rects[0],  # 바운딩 박스 좌표 저장
                        "image_type": False  # 기본적으로 False (추후 판단)
                    })

            pdf_data.append({"page": page_num + 1, "text": text, "images": image_data})

        self.pdf_data = pdf_data
        return pdf_data

    def extract_image_bytes(self, doc, xref):
        """
        PDF에서 이미지 바이트 데이터를 추출
        """
        pix = fitz.Pixmap(doc, xref)
        if pix.n > 4:  # CMYK를 RGB로 변환
            pix = fitz.Pixmap(fitz.csRGB, pix)
        return pix.tobytes()

    def convert_image_to_base64(self, image_bytes):
        """
        이미지 바이트를 base64 문자열로 변환
        """
        return base64.b64encode(image_bytes).decode('utf-8')

# 이미지 분석 & 필터링 (PDFProcessor 상속)
class ImageAnalyzer(PDFProcessor):
    def __init__(self, pdf_path: str):
        super().__init__(pdf_path)
        self.llm = ChatOpenAI(model="gpt-4o", max_tokens=1000)

    def calculate_image_area_ratio(self, image_bbox, page_width, page_height):
        """
        페이지에서 감지된 이미지의 면적 비율을 계산 (ImageAnalyzer 내부로 이동)
        """
        x0, y0, x1, y1 = image_bbox
        image_area = (x1 - x0) * (y1 - y0)
        page_area = page_width * page_height
        return image_area / page_area if page_area > 0 else 0

    def classify_image_type(self, image_bytes):
        """
        GPT-4o를 사용하여 이미지가 그래프/도표인지 판별
        """
        base64_image = self.convert_image_to_base64(image_bytes)

        prompt = "다음 이미지를 그래프/도표인지 판단하세요. '그래프/도표' 또는 '그외' 중 하나만 출력하세요."
        messages = [
            SystemMessage(content="당신은 이미지 분류 전문 AI 에이전트입니다."),
            HumanMessage(content=[
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ])
        ]

        response = self.llm.invoke(messages)
        return response.content.strip() == "그래프/도표"

    def should_interpret_image(self, image_bytes, image_bbox, page_width, page_height, threshold=0.5):
        """
        이미지가 도표이거나 면적 비율이 클 경우 True 반환
        """
        area_ratio = self.calculate_image_area_ratio(image_bbox, page_width, page_height)
        return area_ratio >= threshold or self.classify_image_type(image_bytes)

    def filter_important_images(self):
        """
        PDF 데이터에서 해석이 필요한 이미지만 필터링하여 반환
        """
        filtered_data = []
        for page in self.pdf_data:
            page_width, page_height = 800, 1000  # PDF 크기 (임시 값)
            important_images = []

            for img in page["images"]:
                if self.should_interpret_image(img["image_bytes"], img["bbox"], page_width, page_height):
                    img["image_type"] = True
                    important_images.append(img)

            filtered_data.append({"page": page["page"], "text": page["text"], "images": important_images})

        return filtered_data


# 발표 대본 생성 (ImageAnalyzer 상속)
class ScriptGenerator(ImageAnalyzer):
    def __init__(self, pdf_path: str):
        super().__init__(pdf_path)
        self.llm_script = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
        self.vision_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
        self.previous_script: List[str] = []

    def image_analysis(self, page_idx: int) -> str:
        """
        현재 페이지의 이미지에 대한 분석 수행
        """
        page_data = self.pdf_data[page_idx]
        image_descriptions = []

        for image in page_data["images"]:
            if image["image_type"]:
                template = """
                다음은 발표 자료에 대한 정보 입니다:
                - 현재 페이지 텍스트: {text}
                - 현재 페이지 이미지: {image}

                이를 바탕으로 이미지에 대한 설명을 작성해주세요.
                """
                prompt = PromptTemplate.from_template(template)
                image_desc = self.vision_llm.invoke(prompt.format(text=page_data["text"], image=image["image"]))
                image_descriptions.append(image_desc)

        return "\n".join(image_descriptions) if image_descriptions else ""

    def generate_script(self, page_idx: int) -> str:
        """
        발표 대본 생성 메서드
        """
        page_data = self.pdf_data[page_idx]
        text = page_data["text"]
        image_descriptions = self.image_analysis(page_idx)

        previous_script_text = "\n".join(self.previous_script[-3:]) if self.previous_script else "없음"

        template = """
        다음은 발표 자료에 대한 정보 입니다:
        - 현재 페이지 텍스트: {text}
        - 현재 페이지 이미지: {image_descriptions}
        - 이전 페이지 발표 대본: {previous_script}

        위 정보를 바탕으로 발표자가 자연스럽게 발표할 수 있도록 대본을 작성해주세요.
        """
        prompt = PromptTemplate.from_template(template)
        script = self.llm_script.invoke(prompt.format(text=text, image_descriptions=image_descriptions, previous_script=previous_script_text))

        self.previous_script.append(script)
        return script

    def generate_full_script(self) -> str:
        """
        전체 발표 대본을 생성하는 메서드
        """
        # PDF 데이터 추출
        self.extract_page()

        # 중요 이미지 필터링
        self.filter_important_images()

        # 각 페이지별로 발표 대본 생성
        all_scripts = []
        for page_idx in range(len(self.pdf_data)):
            script = self.generate_script(page_idx)
            all_scripts.append(f"Page {page_idx + 1}:\n{script}\n")

        # 전체 발표 대본 반환
        return all_scripts
